<a href="https://colab.research.google.com/github/20241984/Ejercicios/blob/main/Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importar pandas
import pandas as pd
import numpy as np
# importar  las librerias de SKlearn
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn import set_config
set_config(display='diagram')


In [ ]:
path = '/content/sample_data/Cereal with missing values.xlsx'
df = pd.read_excel(path)
df.head()

,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2,2.0,180.0,1.5,10.5,10.0,70,25.0,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3,2.0,NaN,2.0,18.0,NaN,100,25.0,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6,2.0,290.0,2.0,17.0,1.0,105,25.0,1,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1,3.0,210.0,0.0,13.0,9.0,45,25.0,2,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3,2.0,140.0,2.0,13.0,7.0,105,25.0,3,1.00,0.50,40.400208


Qué tan bien se pueden predecir las calorías basándose en el:
 fabricante
 tipo de cereal,
 gramos de grasa,
  gramos de azúcares
  peso en onzas por una porción del cereal?

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

In [ ]:
# Definir caraxteristica y objetivo
feature_columns = ["Manufacturer", "type", "grams of fat","grams of sugars", "Weight in ounces per one serving"]
target_column = ["calories per serving"]

X =df[feature_columns]
y =df[target_column]
X.head()


,Manufacturer,type,grams of fat,grams of sugars,Weight in ounces per one serving
0,General Mills,Cold,2.0,10.0,1.00
1,General Mills,Cold,2.0,NaN,1.33
2,General Mills,Cold,2.0,1.0,1.00
3,General Mills,Cold,3.0,9.0,1.00
4,General Mills,Cold,2.0,7.0,1.00


In [ ]:
# Importen train test split de sklearn
from sklearn.model_selection import train_test_split
# Utilizar train test split en los datos para prepararlos para aprendizaje automático.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
#Identificar cada característica como numérica, ordinal o nominal.
X_train.head()
# (Entreguen esta pregunta en una celda de texto en un cuaderno de Colab).

,Manufacturer,type,grams of fat,grams of sugars,Weight in ounces per one serving
9,General Mills,Cold,1.0,10.0,1.0
5,General Mills,Cold,1.0,13.0,1.0
34,Kelloggs,NaN,1.0,NaN,1.3
22,Kelloggs,Cold,1.0,NaN,1.0
30,Kelloggs,Cold,0.0,11.0,1.0


Numericas:
grams of fat
grams of sugars
Weight in ounces per one serving
Nominales
Manufacturer y type



In [ ]:
# Utilizar pipelines y transformadores de columnas para completar las siguientes tareas:
# 1 Imputar algún valor faltante. Utilizar la estrategia “mean” para las columnas numéricas

# y la estrategia “most_frequent” para las columnas categóricas.
# 2 Realizar una codificación one-hot a las características nominales.
# Escalar las columnas numéricas.
numeric_transformer = Pipeline(
    steps = [('imputer', SimpleImputer(strategy='mean')),
          ('scaler', StandardScaler())
          ])
numeric_transformer

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot',OneHotEncoder(sparse=False, handle_unknown='ignore'))
          ])
categorical_transformer

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [ ]:
# Todos los datos de preprocesamiento deben estar contenidos
# en un único objeto de preprocesamiento.
preprocessor = ColumnTransformer(
        transformers = [
            ("num", numeric_transformer,["grams of fat", "grams of sugars","Weight in ounces per one serving"]),
            ("cat", categorical_transformer,["Manufacturer","type"]),
            ]
        )
preprocessor


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['grams of fat', 'grams of sugars',
                                  'Weight in ounces per one serving']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Manufacturer', 'type'])])

In [20]:
# Utilizar el objeto de preprocesamiento para transformar los datos de manera adecuada,
# evitar la fuga de datos para dejarlos listos para el modelado.
# Muestren los arrays de NumPy resultantes.
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
X_train_processed
X_test_processed



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 1.0082989 ,  0.04341517, -0.15718492,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 1.0082989 ,  1.42401746,  3.14911856,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.        , -0.87698636, -0.15718492,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 1.0082989 ,  0.73371631, -0.15718492,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.        , -0.18668522, -0.15718492,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-1.0082989 , -1.56728751, -0.15718492,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.   